# Cleanup
Konvertering af brevene i json-format til en enkelt csv-fil.

Vi renser ud i de mest basale tekstformateringer, såsom linjeskift, tabuleringer og dobbelt mellemrum.

Omdøber kolonnerne og konverterer datoerne til datetime-format og introducerer et id.


In [6]:
# load jsonfile from disk
import json
import pandas as pd


In [7]:

with open('data/letters.json', 'r') as f:
    letters = json.load(f)

from MLStripper import strip_tags

# strip html tags from text
for letter in letters:
    letter['Text'] = strip_tags(letter['Text'])

df = pd.DataFrame(letters)

# lowercase all column names
df.columns = map(str.lower, df.columns)


In [8]:
# Text cleanup

# replace crlf with lf
df['text'] = df['text'].str.replace('\r\n', '\n')
# replace leading and trailing whitespace
df['text'] = df['text'].str.strip()

# replace single newlines with space
df['text'] = df['text'].str.replace('\n', ' ')

# replace multiple newlines with single newline
df['text'] = df['text'].str.replace('\n+', '\n')

# replace \xa0 with space
df['text'] = df['text'].str.replace('\xa0', ' ')

# replace \xa with ''
df['text'] = df['text'].str.replace('\xad', '')
df['text'] = df['text'].str.replace('\x95', '')

# replace multiple spaces with single space
df['text'] = df['text'].str.replace('\s+', ' ')


In [9]:
# Dates
df['date'] = pd.to_datetime(df['letterdate'])
df['date'] = df['date'].dt.date
df.drop(['letterdate'], axis=1, inplace=True)

# for each letter we calculate the time between the letters
from datetime import datetime
dates = df['date'].tolist()
#dates = [datetime.strptime(date, '%Y-%m-%d') for date in dates]

# calculate the time between the letters
from datetime import timedelta
time_between = [0.0]
for i in range(len(dates) - 1):
    time_between.append((dates[i + 1] - dates[i]).days)

df["days_from_last"] = time_between



In [10]:
# order letters by date
df.sort_values(by=['date'], inplace=True)

# Add an id column as the first column
df.insert(0, 'id', range(1, 1 + len(df)))

#
df.to_csv('data/letters.csv', index=False)